<a href="https://colab.research.google.com/github/SarahVSargent/stock_signals_regressions/blob/main/Fama_FrenchThreeFactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Intro

In [ ]:
#coonnect google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Import tools
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

!pip install pandasql

import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats.mstats import winsorize
import math
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import pandasql as ps

In [ ]:
file = "/content/drive/MyDrive/Exeter google drive /internships jobs/FIMA '23/Fama-french 5 factor/CRSPMonthly1990Through2022.csv"
CRSPMonthly1990Through2022 = pd.read_csv(file)
print(CRSPMonthly1990Through2022.columns)

file = "/content/drive/MyDrive/Exeter google drive /internships jobs/FIMA '23/Fama-french 5 factor/FamaFrenchFiveFactorsMonthly.csv"
FamaFrenchFiveFactorsMonthly = pd.read_csv(file)
print(FamaFrenchFiveFactorsMonthly.columns)

file = "/content/drive/MyDrive/Exeter google drive /internships jobs/FIMA '23/Fama-french 5 factor/ccmfund1990to2022.csv"
ccmfund1990to2022 = pd.read_csv(file)
print(ccmfund1990to2022.columns)

print("imported")

clean just CRSP

In [ ]:
#4:53

#only keeping these relevant variables
CRSPMonthly1990Through2022= CRSPMonthly1990Through2022[["PERMO", "date", "RET", "SHROUT", "ALTPRC", "EXCHCD", "SHRCD", "SICCD", "DLRET", "DLSTCD"]]
#drop duplicates (instead of slect distinct)
CRSPMonthly1990Through2022.drop_duplicates(inplace=True)

In [ ]:
#5:10

#convert variables to numeric- nessesary? Yes, good practice
CRSPMonthly1990Through2022['PERMO']= pd.to_numeric(CRSPMonthly1990Through2022['PERMO'], errors= 'coerce')
CRSPMonthly1990Through2022['date']= pd.to_numeric(CRSPMonthly1990Through2022['date'], format= '%Y%m%d')
CRSPMonthly1990Through2022['RET']= pd.to_numeric(CRSPMonthly1990Through2022['RET'], errors= 'coerce')
CRSPMonthly1990Through2022['SHROUT']= pd.to_numeric(CRSPMonthly1990Through2022['SHROUT'], errors= 'coerce')
CRSPMonthly1990Through2022['ALTPRC']= pd.to_numeric(CRSPMonthly1990Through2022['ALTPRC'], errors= 'coerce')
CRSPMonthly1990Through2022['EXCHCD']= pd.to_numeric(CRSPMonthly1990Through2022['EXCHCD'], errors= 'coerce')
CRSPMonthly1990Through2022['SHRCD']= pd.to_numeric(CRSPMonthly1990Through2022['SHRCD'], errors= 'coerce')
CRSPMonthly1990Through2022['SICCD']= pd.to_numeric(CRSPMonthly1990Through2022['SICCD'], errors= 'coerce')
CRSPMonthly1990Through2022['DLRET']= pd.to_numeric(CRSPMonthly1990Through2022['DLRET'], errors= 'coerce')
CRSPMonthly1990Through2022['DLSTCD']= pd.to_numeric(CRSPMonthly1990Through2022['DLSTCD'], errors= 'coerce')

#convert returns to percent
CRSPMonthly1990Through2022['RET'] *=100
CRSPMonthly1990Through2022['DLRET'] *=100


In [ ]:
#5:30

#only data rows after 2018
CRSP2018to2022= CRSPMonthly1990Through2022[CRSPMonthly1990Through2022['date']>= "2018-01-01"]

#only data rows in US
#what numbers mean?- only index
CRSP2018to2022= CRSP2018to2022[(CRSP2018to2022['SHRCD']==10) | (CRSP2018to2022['SHRCD']==11)]

In [ ]:
#calculate market cap (millions) per row/ calculated column/ new column/ add column
#market cap is total value, shrout is SHaRes OUTstanding, altprc
CRSP2018to2022['market_cap']= abs(CRSP2018to2022['SHROUT']*CRSP2018to2022['ALTPRC'])/1000
#explain this?, inplace not there before
#replacing np.nan (null) rows with 0, will drop later on
CRSP2018to2022['market_cap'].replace(0, np.nan, inplace=True)

In [ ]:
#5:56
#add exchange column
#define exchange labels and keep only relevant labels

#1 or 31, traded on NYSE, etc.
CRSP2018to2022['exchange'] = ""
for i in CRSP2018to2022.index:
  if CRSP2018to2022['EXCHCD'][i] in [1, 31]:
    CRSP2018to2022['exchange'][i] = "NYSE"
  elif CRSP2018to2022['EXCHCD'][i] in [2, 32]:
    CRSP2018to2022['exchange'][i] = "AMEX"
  elif CRSP2018to2022['EXCHCD'][i] in [3, 33]:
    CRSP2018to2022['exchange'][i] = "NASDAQ"
  else:
    CRSP2018to2022['exchange'][i] = "Other"
#filter out exchanges of other type ?. ! means inequality. So, keep everything but other
CRSP2018to2022= CRSP2018to2022[CRSP2018to2022['exchange'] != "Other"]


In [ ]:
#6:30
#add adjusted return column
#creating new column, all values null
#np.nan is for numbers, as opposed to blank
CRSP2018to2022['ret_adj']= np.nan
#i is variable
for i in CRSP2018to2022.index:
#[column][row],
  if pd.isnull(CRSP2018to2022['DLSTCD'][i]):
    CRSP2018to2022['ret_adj'][i] = CRSP2018to2022['ret'][i]
  elif(pd.notnull(CRSP2018to2022['DLSTCD'][i])) and (pd.notnull(CRSP2018to2022['DLRET'][i])):
    CRSP2018to2022['ret_adj'][i] = CRSP2018to2022['DLSTCD'][i]
#what numbers range mean- index, defined in codebook
  elif(551<=CRSP2018to2022['DLSTCD'][i]<=574) or (CRSP2018to2022['DLSTCD'][i] in [500,520,580,584]):
    CRSP2018to2022['ret_adj'][i] = -30
  else:
    CRSP2018to2022['ret_adj'][i] = -100



In [ ]:
#6:40
#drop unneeded collumns, neater dataframe

#no need for pandas, only sequel
#CRSP2018to2022= CRSP2018to2022.reset_index()

#axis 0 is certain row (implicit), axis 1 is certain columns (have to make explicit)
CRSP2018to2022.drop(['SHRCD', 'DLRET', 'DLSTCD'], inplace=True, axis=1)

print(CRSP2018to2022)
print(CRSP2018to2022.columns)


NameError: ignored

clean CCM data

In [ ]:
#only keeping these relevant variables
ccmfund1990to2022= ccmfund1990to2022[["GVKEY", "LPERMINO", "datadate", "LINKTYPE", "LINKENDDT", "seq", "ceq", "at", "lt", "txditc","txdb", "itcb", "pstkrv", "pstkl", "pstk", "indfmt", "datafmt"]]
#drop duplicates (instead of slect distinct)
ccmfund1990to2022.drop_duplicates(inplace=True)

In [ ]:
#7:57

ccmfund1990to2022['GVKEY']= pd.to_numeric(ccmfund1990to2022['GVKEY'], errors= 'coerce')
ccmfund1990to2022['LPERMINO']= pd.to_numeric(ccmfund1990to2022['LPERMINO'], errors= 'coerce')
ccmfund1990to2022['datadate']= pd.to_numeric(ccmfund1990to2022['datadate'], format= '%Y%m%d')
ccmfund1990to2022['LINKTYPE']= ccmfund1990to2022['LINKTYPE'].apply(str)
ccmfund1990to2022['LINKENDDT']= pd.to_numeric(ccmfund1990to2022['LINKENDDT'], format= '%Y%m%d')
ccmfund1990to2022['seq']= pd.to_numeric(ccmfund1990to2022['seq'], errors= 'coerce')
ccmfund1990to2022['ceq']= pd.to_numeric(ccmfund1990to2022['ceq'], errors= 'coerce')
ccmfund1990to2022['at']= pd.to_numeric(ccmfund1990to2022['at'], errors= 'coerce')
ccmfund1990to2022['lt']= pd.to_numeric(ccmfund1990to2022['lt'], errors= 'coerce')
ccmfund1990to2022['txditc']= pd.to_numeric(ccmfund1990to2022['txditc'], errors= 'coerce')
ccmfund1990to2022['txdb']= pd.to_numeric(ccmfund1990to2022['txdb'], errors= 'coerce')
ccmfund1990to2022['itcb']= pd.to_numeric(ccmfund1990to2022['itcb'], errors= 'coerce')
ccmfund1990to2022['pstkrv']= pd.to_numeric(ccmfund1990to2022['pstkrv'], errors= 'coerce')
ccmfund1990to2022['pstkl']= pd.to_numeric(ccmfund1990to2022['pstkl'], errors= 'coerce')
ccmfund1990to2022['pstk']= pd.to_numeric(ccmfund1990to2022['pstk'], errors= 'coerce')

ccmfund1990to2022['indfmt']= ccmfund1990to2022['indfmt'].apply(str)
ccmfund1990to2022['datafmt']= ccmfund1990to2022['datafmt'].apply(str)


NameError: ignored

In [ ]:
#8:04

#only data rows after 2018
ccmfund2018to2022= ccmfund1990to2022[ccmfund1990to2022['date']>= "2018-01-01 00:00:00"]

#only keep data in correct format
ccmfund1990to2022= ccmfund1990to2022[ccmfund1990to2022['indmt']=="INTL"]
ccmfund1990to2022= ccmfund1990to2022[ccmfund1990to2022['datafmt']=="STD"]

#only keep valid links
ccmfund1990to2022= ccmfund1990to2022[(ccmfund1990to2022['LINKTYPE']=="LU")| (ccmfund1990to2022['LINKTYPE']== "LC")]

#only keep links active at datadate
ccmfund1990to2022= ccmfund1990to2022[(ccmfund1990to2022['datadate']<= ccmfund1990to2022['LINKENDDT'])| pd.isnull(ccmfund1990to2022['LINKENDDT'])]


In [ ]:
#8:42

#book value calculated column

#?replace or vs coalesce
#ccmfund1990to2022['book_value']= abs(ccmfund1990to2022['seq']*CRSP2018to2022['ALTPRC'])
#df['A'].combine_first(df['B'])
#df['c'] = np.where(df["a"].isnull(), df["b"], df["a"] )



#variable a
#variable b
#variable c

ccmfund1990to2022['book_value']= ccmfund1990to2022.ccmfund1990to2022['seq'].combine_first(ccmfund1990to2022.b)


#explain this?, inplace not there before
#replacing np.nan (null) rows with 0, will drop later on
ccmfund1990to2022['book_value'].replace(0, np.nan, inplace=True)


In [ ]:
#calculated column for operating profitability op

#variable a
#variable b
#variable c


In [ ]:
#8:57

#keep only the last observation of each year
ccmfund1990to2022['datadate']= pd.to_datetime(ccmfund1990to2022['datadate'])
ccmfund1990to2022['year']= ccmfund1990to2022['datadate'].dt.year
ccmfund1990to2022= ccmfund1990to2022.sort_values(by= ['PERMO', "datadate"], ascending= True)

###????
ccmfund1990to2022.drop(['SHRCD', 'DLRET', 'DLSTCD'], inplace=True, axis=1)

print(CRSP2018to2022)
print(CRSP2018to2022.columns)


In [ ]:
#9:15
#drop missing/infinite values

ccmfund1990to2022.replace([np.inf, -np.inf], np.nan, inplace=True)
ccmfund1990to2022= ccmfund1990to2022.dropna()



In [ ]:
#9:21
#add reference date column (for matching; june 1 6/1 of next calander year)
ccmfund1990to2022['year']= pd.to_numeric(ccmfund1990to2022['year'], errors= 'coerce')
ccmfund1990to2022['reference_date']= ccmfund1990to2022['year'] +1
ccmfund1990to2022['reference_date']= ccmfund1990to2022['reference_date'].apply(str)
#add year to string
ccmfund1990to2022['reference_date']+= "-06-01"
ccmfund1990to2022['reference_date']= pd.to_datetime(ccmfund1990to2022['reference_date'])
ccmfund1990to2022.drop('year', inplace= True, axis=1 )

print(ccmfund1990to2022)
print(ccmfund1990to2022.columns)

CRSP #CRSP2018to2022 AND COMPUSTAT #ccmfund1990to2022

In [ ]:
#9:44
#parse relevant CRSP variables and store as stocks dataframe

#only keeping these relevant variables
#substitute for select distinct
stonks2018To2022= CRSPMonthly1990Through2022[["PERMO", "date", "ret_adj","market_cap"]]
#drop duplicates (instead of select distinct)
stonks2018To2022.drop_duplicates(inplace=True)

#drop infinite/missing values
stonks2018To2022.replace([np.inf, -np.inf], np.nan, inplace=True)
stonks2018To2022= stonks2018To2022.dropna()


In [ ]:
#9:55
#add reference date to merdge stonks and cmmfund

stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
stonks2018To2022['reference_date']= ""
for i in stonks2018To2022.index:
  if stonks2018To2022['date'][i].month<6:
    stonks2018To2022['reference_date'][i]= str(stonks2018To2022['date['][i].year-1)+"-06-01"
  else:
    stonks2018To2022['reference_date'][i]= str(stonks2018To2022['date['][i].year)+"-06-01"
stonks2018To2022['reference_date']= pd.to_datetime(stonks2018To2022['reference_date'])


In [ ]:
#10:16

#merdge ccm dataframe into stoncks dataframe
#left join
ccmfund1990to2022.join(stonks2018To2022, on= 'permo' and 'reference_date', how='left')
#left_df.merge(right_df.rename({'user_id': 'user_id_r'}, axis=1),left_on='user_id', right_on='user_id_r', how='left')
#what's the name of df now?

df.drop('index', inplace= True, axis=1)

In [ ]:
#10:31

#add market equity (for small vs big)

stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
#only taking dates in the month of december?
stocksMarketEquity= stonks2018To2022[stonks2018To2022['date'].dt.month ==12]
stocksMarketEquity['reference_date']= stonks2018To2022['date'].dt.year +1
stocksMarketEquity['reference_date']= stonks2018To2022['reference_date'].apply(str)
#add year to string
stocksMarketEquity['reference_date']+= "-06-01"
stocksMarketEquity['reference_date']= pd.to_datetime(stocksMarketEquity['reference_date'])

#parse/ only selecting relevant variables
stocksMarketEquity= stocksMarketEquity[["PERMO", "reference_date", "market_cap"]]
#rename column market cap to market equity
stocksMarketEquity.rename(columns={'market_cap': 'market_equity'}, inplace=True)
#drop duplicates (instead of select distinct)
stocksMarketEquity.drop_duplicates(inplace=True)


#left join marketequity to stonks
stonks2018To2022.merge(stocksMarketEquity.rename({'PERMO': 'PERMO'}, axis=1),left_on='PERMO', right_on='PERMO', how='left')

#clean stonks
stonks2018To2022.drop('index', inplace= True, axis= 1)
stonks2018To2022= stonks2018To2022.dropna()

In [ ]:
#11:03
#add book to market and market cap columns (for growth neutral value)


stonks2018To2022['bm_ratio']= stonks2018To2022['book_value']/ stonks2018To2022['market_equity']

#add market cap of june of year y of reference data
stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
#only taking dates in the month of december?
stocksWeight= stonks2018To2022[stonks2018To2022['date'].dt.month ==6]

##parse/ only selecting relevant variables
#stocksMarketEquity= stocksMarketEquity[["PERMO", "reference_date", "market_cap"]]
##rename column market cap to market equity
#stocksMarketEquity.rename(columns={'market_cap': 'market_equity'}, inplace=True)
#drop duplicates (instead of select distinct)
stocksWeight.drop_duplicates(inplace=True)


#left join marketequity to stonks?
stonks2018To2022.merge(stocksWeight.rename({'PERMO': 'PERMO'}, axis=1),left_on='PERMO', right_on='PERMO', how='left')

#rename column
stonks2018To2022.rename(columns={'market_cap': 'market_cap_equity'}, inplace=True)

#clean stonks
stonks2018To2022.drop('index', inplace= True, axis= 1)


In [ ]:
#11:11

#quick filter of null and infiite, only keep dates after 2020 (OR 2018???)

stonks2018To2022.replace([np.inf, -np.inf], np.nan, inplace=True)
stonks2018To2022= stonks2018To2022.dropna()

stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
stonks2018To2022= stonks2018To2022[stonks2018To2022['date'] >= '2020-01-01 00:00:00']


Factor: smb sorting

In [ ]:
#11:29
#only data in june

stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
sizeBreakpoints= stonks2018To2022[(stonks2018To2022['date'].dt.month== 6) & (stonks2018To2022['exchange']== "NYSE")]
sizeBreakpoints= sizeBreakpoints['PERMO','reference_date','market_cap']
sizeBreakpoints['reference_date']= pd.to_datetime(sizeBreakpoints['reference_date'])
g= sizeBreakpoints.groupby(['reference_date'])




In [ ]:
#11:51
#create size sorts and add median size column


#should i be using sizebreakpoints?
sizesorts= stonks2018To2022[(stonks2018To2022['date'].dt.month== 6)]

sizesorts['size_median']=""
#median or .quantile(0.5)
for i in sizesorts.index:
    sizesorts['size_median'][i]= g.get_group(sizesorts['reference_date'][i]).reset_index()['market_cap'].median()



In [ ]:
#12:15
#sort firms based on size

sizesorts["size_portfolio"]= ""
for i in sizesorts.index:
  if sizesorts["market_cap"][i] > sizesorts['size_median'][i]:
    sizesort[i]["size_portfolio"]= "B"
  else:
    sizesort[i]["size_portfolio"]= "S"


In [ ]:
#add size sorts data into stocks

#drop duplicates (instead of select distinct)
sizesort.drop_duplicates(inplace=True)

#left join marketequity to stonks?
stonks2018To2022.merge(sizesort.rename({'reference_date': 'reference_date', 'PERMO':'PERMO'} , axis=1),left_on='PERMO', right_on='PERMO', how='left')


Value factor sort

In [ ]:
#12:35
#only data in june

stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
valueBreakpoints= stonks2018To2022[(stonks2018To2022['date'].dt.month== 6) & (stonks2018To2022['exchange']== "NYSE")]
valueBreakpoints= valueBreakpoints['PERMO','reference_date','market_cap']
sizeBreakpoints['reference_date']= pd.to_datetime(sizeBreakpoints['reference_date'])
g= sizeBreakpoints.groupby(['reference_date'])


In [ ]:
#12:47
#calculate 30th and 70th percentiles


#should i be using sizebreakpoints?
valuesorts= stonks2018To2022[(stonks2018To2022['date'].dt.month== 6)]

#creation of variables?
valuesorts['value_q30']=""
valuesorts['value_q70']=""
#explain this
for i in sizesorts.index:
    valuesorts['value_q30'][i]= g.get_group(g.get_group(valuesorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.3)
    valuesorts['value_q70'][i]= g.get_group(g.get_group(valuesorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.7)




In [ ]:
#12:53
#sort based on value, neatral, growth (H, M, L)

valuesorts['value_portfolio']=""
for i in valuesorts.index:
    if valuesorts['bm_ratio'][i]>= valuesorts['value_q70'][i]:
       valuesorts['value_portfolio'][i]= "H"
    elif valuesorts['bm_ratio'][i]> valuesorts['value_q30'][i]:
       valuesorts[i]["value_portfolio"]= "L"
    else:
         valuesorts[i]["value_portfolio"]= "M"

In [ ]:
#13:01
#add size sorts data into stocks

#drop duplicates (instead of select distinct)
valuesorts.drop_duplicates(inplace=True)

#left join marketequity to stonks?
stonks2018To2022.merge(valuesorts.rename({'reference_date': 'reference_date', 'PERMO':'PERMO'} , axis=1),left_on='PERMO', right_on='PERMO', how='left')


merdging dataframes and factors

In [ ]:
#13:25
#new groupby variable for avg weighted returns
stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])

valweightedret= stonks2018To2022.groupby(['date', 'size_portfolio', 'value_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))


In [ ]:
#14:01
#create factors dataframe using only only

factors=stonks2018To2022['date']
factors.drop_duplicates(inplace=True)
factors['date']= pd.to_datetime(factors['date'])


In [ ]:
#14:09
#add in three factors smb, hml, mkt ret

factors['smb']=""
#what is index?
for i in factors.index:
  dateofdata= factors['date'][i]
  factors['smb'][i]= np.average(valweightedret[dateofdata]["S"])- np.average(valweightedret[dateofdata]["B"])
#what does groupby do and what does line mean
#does order matter?
valweightedret= stonks2018To2022.groupby(['date', 'value_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))

factors['hml']=""
for i in factors.index:
  dateofdata= factors['date'][i]
  factors['hml'][i]= np.average(valweightedret[dateofdata]["H"])- np.average(valweightedret[dateofdata]["L"])

factors['mkt_ret']=""
weightedavgmonthlyret= stonks2018To2022.groupby(['date']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))
for i in factors.index:
  dateofdata= factors['date'][i]
  factors['mkt_ret'][i]= weightedavgmonthlyret[dateofdata]



In [ ]:
#14:43
#clean

#Take out last month- ret= 100% due to missing dlret data??
factors['date']= pd.to_datetime(factors[['date']])
factors= factors[factors['date']< '2022-03-31']

factors.drop_duplicates(inplace=True)

#substitute for select distinct
#why not including company name?
factors= factors[["date", "mkt_ret", "smb","hml"]]


In [ ]:
#15:00
#ensure factors sorted by date
factors['date']= pd.to_datetime(factors[['date']])
factors= factors.sort_values(by=['date'], ascending= True)


Clean ff data

In [ ]:
#15:12
#import fama-french data

print(FamaFrenchFiveFactorsMonthly)


NameError: ignored

In [ ]:
#15:32

#rename column Date to date, etc
FFFFmonthly=FamaFrenchFiveFactorsMonthly

FFFFmonthly.rename(columns={'Date': 'date', "Mkt_RF - RF": "mkt_ff", "SMB": "smb_ff", "HML": "hml_ff"}, inplace=True)


In [ ]:
#15:35
#convert variables

FFFFmonthly[mkt_ff]= pd.to_numeric(FFFFmonthly[mkt_ff] , errors='coerce')
FFFFmonthly[smb_ff]= pd.to_numeric(FFFFmonthly[smb_ff] , errors='coerce')
FFFFmonthly[hml_ff]= pd.to_numeric(FFFFmonthly[hml_ff] , errors='coerce')


In [ ]:
#15:42
#add month and year columns to factors and FFFFmonthly
FF=FFFFmonthly

factors['date']= pd.to_datetime(factors[['date']])
factors['month']= factors['date'].dt.month
factors['year']= factors['year'].dt.year

FF['month']= ""
for i in FF.index:
  #take last two digits of date (ie take the month)
  FF['month'][i]= int(str(FF['date'][i][-2]+ str(FF['date'][i][-1])))


FF['year']= ""
for i in FF.index:
  #int(str) function takes the string and turns it into an integer
  #take first 4 digits of date (ie take the year characters)
  FF['year'][i]= int(str(FF['date'][i][0:4]))


In [ ]:
#15:55
#merdge FF INTO factors

#order correct?
factors.join(FF, on= 'month' and 'year', how='left')

print(factors)


FF vs factors

In [ ]:
#16:24

#various percentiles print
factors.describe(percentiles=[0.1, .25, .5, .75, .9])



In [ ]:
#16:30
#KS tests
#KS tests (one-sample) if a dataset has a normal distrobution,
 #(two-sample) if two datasets have similar distrobutions
 #results are (d,f), d being the maximum difference in values, other is p-value

ks_2samp(factors['mkt'], FF['mkt_ff'])


ks_2samp(factors['smb'], FF['smb_ff'])


ks_2samp(factors['hml'], FF['hml_ff'])



In [ ]:
#16:44
#compute correlation

factors.corr(method='pearson')





MODS to three factor

In [ ]:
#merdge crsp and compustat data

In [ ]:
#2:58
#clean CRSP/compustat merged datafram

#parse? relevant variables

In [ ]:
#3:11
#convert variables pd.to_numeric

In [ ]:
#calculate vales for each collum (make calulated collum?)

In [ ]:
#table 1
#summary stat for each

In [ ]:
#table 2
#correlatin table for all factors